## Challenge code
 
 In the code below, you are given a few functions:
 
 - `calculate_timbit`: This function will return a timbit associated to the operator $U$ and input state $\rho_0$, given an initial guess  $\rho$ for the timbit. It returns the density matrix representation of the timbit $\rho^{*}.$ **You must complete this function**.
 - `apply_timbit_gate`:  Returns the output density matrix after applying a timbit gate to a state $\rho_0$.
     The input and output density matrices are associated with the first qubit.
 - `SAT`: uses a timbit gate to solve the
   satisfiability problem for an arbitrary Boolean function $f$ (on
   `n_bits` bits) with an oracle in matrix form `U_f`, using `q` timbit gates, and `rho` being the initial guess for the NP fixed point. The output should be the computational basis measurement probabilities for the last qubit, which should be `[1. , 0.]` if and only if there are no elements $x$ such that $f(x) =1.$ **You must complete this function**.

![img](images/timbit.png)
 
 ### Inputs
 
 As input to this problem, you are given:
 
 - `q (int)`: the number of times the timbit gate is applied to solve the SAT problem.
 
 Additionally, you are given the following global variables:
 
 - `U_f array(array(float))`:  the oracle $U_f$ we will use to test your solution in matrix form.
 - `U_NP array(array(float))`: the gate $U_{\textsf{NP}}$ as defined above.
 - `rho (array(array(float))`: the initial guess for the stationary state of the timbit gate $\textsf{T}_{\textsf{NP}}.$
 
 ### Output
 
 The output for this challenge corresponds to the output of your `SAT` function. It should produce a `numpy.tensor` of length 2 corresponding to the measurement probabilities in the computational basis for the qubit on which the timbit gates are applied.
 ### Imports
 The cell below specifies the libraries you should use in this challenge. Run the cell to import the libraries. ***Do not modify the cell.***

In [1]:
import json
import pennylane as qml
from pennylane import numpy as np
import scipy
import sympy as sp

### Code
 Complete the code below. Note that during QHack, some sections were not editable. We've marked those sections accordingly here, but you can still edit them if you wish.

In [ ]:
# magic gate that is going to find rho^*
U_NP = [[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0]]

sp.Matrix(U_NP)

Matrix([
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 1, 0, 0],
[0, 0, 1, 0]])

In [8]:
def calculate_timbit(U, rho_0, rho, n_iters):
    """
    This function will return a timbit associated to the operator U and a state passed as an attribute.

    Args:
        U (numpy.tensor): A 2-qubit gate in matrix form.
        rho_0 (numpy.tensor): The matrix of the input density matrix.
        rho (numpy.tensor): A guess at the fixed point C[rho] = rho.
        n_iters (int): The number of iterations of C.

    Returns:
        (numpy.tensor): The fixed point density matrices.
    """
    # this rho is going to converge to the fixed point rho^*
    dev = qml.device("default.mixed", wires=2)
    @qml.qnode(dev)
    def C_U(U, rho_0, rho):
        qml.QubitDensityMatrix(rho_0, wires=[0])
        qml.QubitDensityMatrix(rho, wires=[1])
        qml.QubitUnitary(U, wires=[0, 1])
        return qml.density_matrix(wires=1)
    
    # reapply the U gate n_times
    # init
    rho_star = C_U(U, rho_0, rho)
    # iterate
    for _ in range(n_iters - 1):
        rho_star = C_U(U, rho_0, rho_star)

    return rho_star

rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
rho_0 = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

display(sp.Matrix(np.round(rho, 5)))

rho_star = calculate_timbit(U_NP, rho_0, rho, 10)
print("converges to (rho_star) :")
display(sp.Matrix(np.round(rho_star, 5)))

Matrix([
[        0.6, 0.1 - 0.1*I],
[0.1 + 0.1*I,         0.4]])

converges to (rho_star) :


Matrix([
[0.6,   0],
[  0, 0.4]])

In [ ]:
def apply_timbit_gate(U, rho_0, timbit):
    """
    Function that returns the output density matrix after applying a timbit gate to a state.
    The density matrix is the one associated with the first qubit.

    Args:
        U (numpy.tensor): A 2-qubit gate in matrix form.
        rho_0 (numpy.tensor): The matrix of the input density matrix.
        timbit (numpy.tensor): The timbit associated with the operator and the state.

    Returns:
        (numpy.tensor): The output density matrices.
    """
    dev = qml.device("default.mixed", wires=2)
    @qml.qnode(dev)
    def T_U(U, rho_0, timbit):
        qml.QubitDensityMatrix(rho_0, wires=[0])
        qml.QubitDensityMatrix(timbit, wires=[1]) # timbit as input instead of rho
        qml.QubitUnitary(U, wires=[0, 1])
        return qml.density_matrix(wires=0) # for qubit 0 instead of 1
    
    return T_U(U, rho_0, timbit)

    
display(sp.Matrix(np.round(rho_0, 5)))

rho_out = apply_timbit_gate(U_NP, rho_0, rho)
sp.Matrix(np.round(rho_out, 5))

Matrix([
[        0.6, 0.1 - 0.1*I],
[0.1 + 0.1*I,         0.4]])

Matrix([
[        0.52, 0.1 - 0.02*I],
[0.1 + 0.02*I,         0.48]])

In [14]:
# inputs:
# U_f: our oracle
# q: number of times we apply the Timbit gate
# rho: initial value of the timbit
# n_bits: number of qubits (this is actually n+1. n plus 1 additional qubit for the output)
def SAT(U_f, q, rho, n_bits):
    """A timbit-based algorithm used to guess if a Boolean function ever outputs 1.

    Args:
        U_f (numpy.tensor): A multi-qubit gate in matrix form.
        q (int): Number of times we apply the Timbit gate.
        rho (numpy.tensor): An initial guess at the fixed point C[rho] = rho.
        n_bits (int): The number of bits the Boolean function is defined on.

    Returns:
        numpy.tensor: The measurement probabilities on the last wire.
    """
    dev = qml.device("default.mixed", wires=n_bits)
    @qml.qnode(dev)
    def H_Uf(U_f, n_bits):
        for i in range(n_bits - 1):
            qml.Hadamard(wires=i)
        qml.QubitUnitary(U_f, wires=range(n_bits))
        return qml.density_matrix(wires=n_bits - 1)
    
    # output of the H_Uf gate
    rho_0 = H_Uf(U_f, n_bits)

    # apply timbit gate q times
    for _ in range(q):
        timbit = calculate_timbit(U_NP, rho_0, rho, 100)
        rho_0 = apply_timbit_gate(U_NP, rho_0, timbit)

    @qml.qnode(dev)
    def calc_probs():
        qml.QubitDensityMatrix(rho_0, wires=n_bits - 1)
        return qml.probs(wires=n_bits - 1)
    
    return calc_probs()


I = np.eye(2)
X = qml.matrix(qml.PauliX(0))

U_f = scipy.linalg.block_diag(I, X, I, I, I, I, I, I)
rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
q = 1
n_bits = 4

print(SAT(U_f, q, rho, n_bits))


[0.78125 0.21875]


In [18]:
# NOW WE ARE GOING TO PERFORM SOME CHECK
# same as before but we output the density matrix

def SAT(U_f, q, rho, n_bits):
    dev = qml.device("default.mixed", wires=n_bits)
    @qml.qnode(dev)
    def H_Uf(U_f, n_bits):
        for i in range(n_bits - 1):
            qml.Hadamard(wires=i)
        qml.QubitUnitary(U_f, wires=range(n_bits))
        return qml.density_matrix(wires=n_bits - 1)

    rho_0 = H_Uf(U_f, n_bits)
    for _ in range(q):
        timbit = calculate_timbit(U_NP, rho_0, rho, 100)
        rho_0 = apply_timbit_gate(U_NP, rho_0, timbit)

    @qml.qnode(dev)
    def calc_dm():
        qml.QubitDensityMatrix(rho_0, wires=n_bits - 1)
        return qml.density_matrix(wires=n_bits - 1)
    
    return calc_dm()

In [21]:
# check: if I have no marked elements, I should get rho_out = (I + Z)/2

# zero marked element
U_f = scipy.linalg.block_diag(I, I, I, I, I, I, I, I)
rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
q = 5
n_bits = 4

# we see that it approaches the (I + Z)/2 matrix
sp.Matrix(SAT(U_f, q, rho, n_bits))

Matrix([
[0.999995332588558, 0],
[                0, 0]])

In [ ]:
# check: if I have one marked element, I should get rho_out = I/2, if q large enough

# one marked element
U_f = scipy.linalg.block_diag(I, X, I, I, I, I, I, I)
rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
q = 5
n_bits = 4

# we see that it approaches the I/2 matrix
sp.Matrix(SAT(U_f, q, rho, n_bits))

Matrix([
[0.500047565490071,                 0],
[                0, 0.499947113598848]])

In [22]:
# many marked element
U_f = scipy.linalg.block_diag(I, X, I, X, I, I, I, X)
rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
q = 5
n_bits = 4

# we see that it approaches the I/2 matrix
sp.Matrix(SAT(U_f, q, rho, n_bits))

Matrix([
[0.499998129736156,                 0],
[                0, 0.499998129736156]])

These functions are responsible for testing the solution. You will need to run the cell below. ***Do not modify the cell.***

In [15]:
def run(test_case_input: str) -> str:

    I = np.eye(2)
    X = qml.matrix(qml.PauliX(0))

    U_f = scipy.linalg.block_diag(I, X, I, I, I, I, I, I)
    rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

    q = json.loads(test_case_input)
    output = list(SAT(U_f, q, rho,4))

    return str(output)

def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
    rho_0 = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

    assert np.allclose(
        solution_output, expected_output, atol=0.01
    ), "Your NP-solving timbit computer isn't quite right yet!"

### Test cases
 Running the cell below will load the test cases. ***Do not modify the cell***.
 - input: 1
 	+ expected output: [0.78125, 0.21875]
 - input: 2
 	+ expected output: [0.65820312, 0.34179687]
 - input: 3
 	+ expected output: [0.550056457, 0.449943542]

In [16]:
test_cases = [['1', '[0.78125, 0.21875]'], ['2', '[0.65820312, 0.34179687]'], ['3', '[0.550056457, 0.449943542]']]

### Solution testing
 Once you have run every cell above, including the one with your code, the cell below will test your solution. Run the cell. If you are correct for all of the test cases, it means your solutions is correct. Otherwise, you need to double check your work. ***Do not modify the cell below.***

In [17]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1'...
Correct!
Running test case 1 with input '2'...
Correct!
Running test case 2 with input '3'...
Correct!
